In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

base_model = "HuggingFaceTB/SmolLM2-360M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [4]:
import json
from datasets import Dataset

file_path = "/content/drive/MyDrive/Datasets/sorachio-identity-dataset.jsonl"

data = []
with open(file_path, "r") as f:
    for idx, line in enumerate(f, start=1):
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"❌ JSON decode error on line {idx}: {e}")
            print("📄 Line content:")
            print(line)
            break

dataset = Dataset.from_list(data)

print(dataset[:2])


{'text': ['<|im_start|>system\nYou are a helpful AI assistant named Sorachio, trained by Izzul Fahmi.<|im_end|>\n<|im_start|>user\nHi there!<|im_end|>\n<|im_start|>assistant\nHi! How can I help you today?<|im_end|>', '<|im_start|>system\nYou are a helpful AI assistant named Sorachio, trained by Izzul Fahmi.<|im_end|>\n<|im_start|>user\nHello<|im_end|>\n<|im_start|>assistant\nHello! How can I assist you today?<|im_end|>']}


In [5]:
from torch.nn import Linear

for name, module in model.named_modules():
    if isinstance(module, Linear):
      print(name)

model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp.gate_proj
model.layers.2.mlp.up_proj
model.layers.2.mlp.down_proj
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.o_proj
model.layers.3.mlp.gate_proj
model.layers.3.mlp.up_proj
model.layers.3.mlp.down_proj
model.layers.4.self_attn.q_proj
model.layers.4.self_attn.k_proj
model.layers.4.self_attn.v_proj
model.layers.4.self_attn.o_proj
model.layers.4.mlp.g

In [6]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 8,683,520 || all params: 370,504,640 || trainable%: 2.3437


In [7]:
def format_prompt(example):
    # Memisahkan berdasarkan <|im_start|> dan <|im_end|>
    text = example["text"]
    messages = []

    # Memisahkan berdasarkan token yang ada
    parts = text.split("<|im_end|>")
    for part in parts:
        if "<|im_start|>system" in part:
            messages.append({"role": "system", "content": part.replace("<|im_start|>system\n", "").strip()})
        elif "<|im_start|>user" in part:
            messages.append({"role": "user", "content": part.replace("<|im_start|>user\n", "").strip()})
        elif "<|im_start|>assistant" in part:
            messages.append({"role": "assistant", "content": part.replace("<|im_start|>assistant\n", "").strip()})

    return tokenizer.apply_chat_template(messages, tokenize=False)

def tokenize(example):
    return tokenizer(format_prompt(example), truncation=True, padding="max_length", max_length=256)

# Tokenisasi dataset
tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/207 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./sorachio-lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    weight_decay=0.01,
    fp16=True,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.657300
20,1.605700
30,1.090500
40,1.025600
50,0.928100
60,0.955300
70,0.920100


TrainOutput(global_step=78, training_loss=1.2666121079371526, metrics={'train_runtime': 53.0371, 'train_samples_per_second': 11.709, 'train_steps_per_second': 1.471, 'total_flos': 308399500984320.0, 'train_loss': 1.2666121079371526, 'epoch': 3.0})

In [9]:
peft_model = model
model = model.merge_and_unload()
output_path = "/content/drive/MyDrive/Sorachio-360M-Chat/models/"

model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

('/content/drive/MyDrive/Sorachio-360M-Chat/models/tokenizer_config.json',
 '/content/drive/MyDrive/Sorachio-360M-Chat/models/special_tokens_map.json',
 '/content/drive/MyDrive/Sorachio-360M-Chat/models/vocab.json',
 '/content/drive/MyDrive/Sorachio-360M-Chat/models/merges.txt',
 '/content/drive/MyDrive/Sorachio-360M-Chat/models/added_tokens.json',
 '/content/drive/MyDrive/Sorachio-360M-Chat/models/tokenizer.json')